# Generative AI Day 2

## Direct Inference using Ollama

In [ ]:
#imports
import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display

### Define constants related to Ollama

In [ ]:
OLLAMA_API = "http://localhost:11434/api/chat"
HEADERS = {"Content-Type": "application/json"}
MODEL = "llama3.2"

In [ ]:
messages = [
    {"role": "user", "content": "Describe some business applications of Generative AI"}
]

In [ ]:
payload={
    "model": MODEL,
    "messages": messages,
    "stream": False
}

In [ ]:
# To make sure model is loaded
!ollama pull llama3.2

In [ ]:
response = requests.post(OLLAMA_API, headers=HEADERS, json=payload)
print(Markdown(response.json()["message"]["content"]))

## Introduction to Ollama package

now we will do same thing but instead of maikng a direct HTTP call we will use elegant ollama python package

In [ ]:
import ollama
response = ollama.chat(model=MODEL, messages=messages)
print(response["message"]["content"])

### Alternative approach - using OpenAI python library to connect to Ollama

In [ ]:
from openai import OpenAI
ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')
response = ollama_via_openai.chat.completions.create(
    model=MODEL,
    messages=messages
)
print(response.choices[0].message.content)

## Now lets complete the same exercise of text summarization using `ollama`

In [ ]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    def __init__(self, url):
        """
            Create this website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(self.url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevent in soup.body(['img', 'style', 'script', 'input']):
            irrelevent.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [ ]:
pmo = Website("https://www.pmindia.gov.in/en/message-from-the-prime-minister/")
print(pmo.title)
print(pmo.text)

In [ ]:
system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [ ]:
def user_prompt_for(website):
    user_prompt = f"You are looking at website titled {website.title}\n"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [ ]:
def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [ ]:
messages_for(pmo)

In [ ]:
def summerize_openai(url):
    website = Website(url)
    ollama_via_openai = OpenAI(base_url="http://localhost:11434/v1", api_key="ollama")
    response = ollama_via_openai.chat.completions.create(
        model=MODEL,
        messages=messages_for(website)
    )
    display(Markdown(response.choices[0].message.content))

In [ ]:
summerize_openai("https://www.pmindia.gov.in/en/message-from-the-prime-minister/")

In [ ]:
def summerize_ollama(url):
    website = Website(url)
    response = ollama.chat(model=MODEL, messages=messages_for(website))
    display(Markdown(response["message"]["content"]))

In [ ]:
summerize_ollama("https://www.pmindia.gov.in/en/message-from-the-prime-minister/")